In [2]:
## STEP 2A — Create Retrieval Documents

import polars as pl

# Load dataset (same as Project 4)
df = pl.read_csv(
    r"E:\__PROJECTS__\resume projects-data analyst\genai-multi-agent-project\data\raw\reviews.csv"
)

reviews_df = (
    df
    .select([
        pl.col("review_topic"),
        pl.col("review"),
        pl.col("score")
    ])
    .with_row_index("review_id")
    .filter(pl.col("review").is_not_null())
)

print("Total reviews:", reviews_df.shape[0])
print(reviews_df.head())

Total reviews: 2438577
shape: (5, 4)
┌───────────┬──────────────┬─────────────────────────────────┬───────┐
│ review_id ┆ review_topic ┆ review                          ┆ score │
│ ---       ┆ ---          ┆ ---                             ┆ ---   │
│ u32       ┆ str          ┆ str                             ┆ f64   │
╞═══════════╪══════════════╪═════════════════════════════════╪═══════╡
│ 0         ┆ ComputerGame ┆ Best 👌 mobile game ever        ┆ 5.0   │
│ 1         ┆ ComputerGame ┆ It's a really good game I'm go… ┆ 5.0   │
│ 2         ┆ ComputerGame ┆ Call of duty mobile is a very … ┆ 5.0   │
│ 3         ┆ ComputerGame ┆ I love the game in it's entire… ┆ 3.0   │
│ 4         ┆ ComputerGame ┆ Unplayable size.... its portab… ┆ 1.0   │
└───────────┴──────────────┴─────────────────────────────────┴───────┘


In [42]:
reviews_df['review_topic'].unique()

review_topic
str
"""ComputerGame"""
"""Music"""
"""Products"""
"""Course"""
"""Food"""


In [3]:
## STEP 2B — Convert Reviews into Retrieval Text Blocks

def build_review_document(row):
    return (
        f"Review Topic: {row['review_topic']}\n"
        f"Score: {row['score']}\n"
        f"Review Text: {row['review']}"
    )

# Sample documents (do NOT embed all 2.4M yet)
sample_docs = (
    reviews_df
    .select(["review_id", "review_topic", "score", "review"])
    .head(5000)  # start small for testing
)

documents = [
    {
        "id": int(row["review_id"]),
        "text": build_review_document(row)
    }
    for row in sample_docs.to_dicts()
]

print("Sample document:")
print(documents[0]["text"])

Sample document:
Review Topic: ComputerGame
Score: 5.0
Review Text: Best 👌 mobile game ever


## STEP 3 — VECTOR STORE CREATION

In [ ]:
%pip install langchain langchain-openai chromadb

In [ ]:
%pip install langchain-community

In [34]:
## Add metadata filtering (core change)
def filter_reviews_by_topic(df: pl.DataFrame, topic: str) -> pl.DataFrame:
    filtered = df.filter(pl.col("review_topic") == topic)

    if filtered.height == 0:
        print("[Retriever] No reviews found for topic. Falling back to full dataset.")
        return df

    print(f"[Retriever] Using {filtered.height} reviews for topic='{topic}'")
    return filtered

In [35]:
## STEP 3A — Create Embeddings + Vector Store

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

# Convert our documents into LangChain Documents
langchain_docs = [
    Document(
        page_content=doc["text"],
        metadata={"review_id": doc["id"]}
    )
    for doc in documents
]

print("Documents prepared for vector store:", len(langchain_docs))


Documents prepared for vector store: 1000


In [ ]:
%pip install sentence-transformers

In [ ]:
%pip install -U langchain-huggingface

In [36]:
## STEP 3B — Build the Vector Store
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2"  # Free, lightweight model
)

vectorstore = Chroma.from_documents(
    documents=langchain_docs,
    embedding=embeddings,
    collection_name="review_rag_test"
)

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 223.65it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [37]:
## STEP 3C — Test Retrieval

query = "complaints about mobile games"

results = vectorstore.similarity_search(query, k=3)

for i, res in enumerate(results, 1):
    print(f"\nResult {i}")
    print(res.page_content[:300])


Result 1
Review Topic: ComputerGame
Score: 5.0
Review Text: The best mobile game in the world

Result 2
Review Topic: ComputerGame
Score: 5.0
Review Text: The best mobile game in the world

Result 3
Review Topic: ComputerGame
Score: 5.0
Review Text: The best mobile game in the world


In [38]:
## STEP 3D — Add HARD NEGATIVE FILTERING
def build_review_document(row):
    sentiment_hint = (
        "Negative review"
        if row["score"] <= 2.0
        else "Neutral review" if row["score"] <= 3.0 else "Positive review"
    )

    return (
        f"Review Type: {sentiment_hint}\n"
        f"Review Topic: {row['review_topic']}\n"
        f"Score: {row['score']}\n"
        f"Review Text: {row['review']}"
    )

In [43]:
# STEP 3E.0 — Define query topic (manual for now)
query_topic = "ComputerGame"

In [44]:
# STEP 3E — Rebuild SAMPLE DOCUMENTS (Topic + Sentiment aware)

topic_df = filter_reviews_by_topic(reviews_df, query_topic)

complaint_docs = (
    topic_df.filter(pl.col("score") <= 2.0)
    .select(["review_id", "review_topic", "score", "review"])
    .head(3000)
)

neutral_docs = (
    topic_df.filter(pl.col("score") == 3.0)
    .select(["review_id", "review_topic", "score", "review"])
    .head(1000)
)

positive_docs = (
    topic_df.filter(pl.col("score") >= 4.0)
    .select(["review_id", "review_topic", "score", "review"])
    .head(1000)
)

sample_docs = pl.concat([complaint_docs, neutral_docs, positive_docs])

# Safety fallback
if sample_docs.height == 0:
    print("[WARN] No documents found after filtering. Falling back to full dataset.")
    sample_docs = reviews_df.select(
        ["review_id", "review_topic", "score", "review"]
    ).head(3000)

# ✅ SINGLE SOURCE OF TRUTH
documents = [
    {
        "id": int(row["review_id"]),
        "score": float(row["score"]),
        "text": build_review_document(row),
    }
    for row in sample_docs.to_dicts()
]

print("Total stratified documents:", len(documents))

[Retriever] Using 50000 reviews for topic='ComputerGame'
Total stratified documents: 5000


In [45]:
## STEP 3F — REBUILD VECTOR STORE (NEW COLLECTION)
'''
⚠️ Use a new collection name to avoid contamination.
'''

langchain_docs = [
    Document(
        page_content=doc["text"],
        metadata={"review_id": doc["id"]}
    )
    for doc in documents
]

vectorstore = Chroma.from_documents(
    documents=langchain_docs,
    embedding=embeddings,
    collection_name="review_rag_complaints_v1"
)

print("New vector store built")

New vector store built


In [33]:
## STEP 3G — RETEST RETRIEVAL (MOST IMPORTANT)
query = "complaints about mobile games"

results = vectorstore.similarity_search(query, k=5)

for i, res in enumerate(results, 1):
    print(f"\nResult {i}")
    print(res.page_content[:400])


Result 1
Review Type: Negative review
Review Topic: ComputerGame
Score: 1.0
Review Text: Too many app crush

Result 2
Review Type: Negative review
Review Topic: ComputerGame
Score: 1.0
Review Text: Too many app crush

Result 3
Review Type: Negative review
Review Topic: ComputerGame
Score: 1.0
Review Text: Too many app crush

Result 4
Review Type: Negative review
Review Topic: ComputerGame
Score: 1.0
Review Text: Too many app crush

Result 5
Review Type: Negative review
Review Topic: ComputerGame
Score: 1.0
Review Text: Too many app crush


In [46]:
## STEP 3H — Split Vector Stores by Sentiment (Safe Version)
negative_docs = [
    Document(
        page_content=doc["text"],
        metadata={"review_id": doc["id"], "score": doc["score"]},
    )
    for doc in documents
    if doc["score"] <= 2.0
]

general_docs = [
    Document(
        page_content=doc["text"],
        metadata={"review_id": doc["id"], "score": doc["score"]},
    )
    for doc in documents
]

print("Negative docs:", len(negative_docs))
print("General docs:", len(general_docs))

# Safety fallback
if len(negative_docs) == 0:
    print("[WARN] No negative documents found. Falling back to general store.")
    negative_docs = general_docs

Negative docs: 3000
General docs: 5000


In [47]:
## create 2 vector stores

negative_vectorstore = Chroma.from_documents(
    documents=negative_docs,
    embedding=embeddings,
    collection_name="review_rag_negative_v1",
)

general_vectorstore = Chroma.from_documents(
    documents=general_docs,
    embedding=embeddings,
    collection_name="review_rag_general_v1",
)

print("Both vector stores created")

Both vector stores created


In [49]:
# STEP 3I — INTENT-AWARE RETRIEVAL (Dynamic k)

query = "complaints about Computer games"

INITIAL_K = 5
MAX_K = 10
SIMILARITY_THRESHOLD = 0.65

# 1️⃣ First pass retrieval
results_with_scores = negative_vectorstore.similarity_search_with_score(
    query, k=INITIAL_K
)

avg_similarity = sum(score for _, score in results_with_scores) / len(
    results_with_scores
)

print(f"[Retriever] Avg similarity @k={INITIAL_K}: {avg_similarity:.3f}")

# 2️⃣ Expand search if similarity is weak
if avg_similarity < SIMILARITY_THRESHOLD:
    print("[Retriever] Low similarity detected. Expanding search...")
    results_with_scores = negative_vectorstore.similarity_search_with_score(
        query, k=MAX_K
    )

# 3️⃣ Final results
final_results = [doc for doc, _ in results_with_scores]

for i, res in enumerate(final_results, 1):
    print(f"\nResult {i}")
    print(res.page_content[:400])

[Retriever] Avg similarity @k=5: 0.840

Result 1
Review Type: Negative review
Review Topic: ComputerGame
Score: 1.0
Review Text: Worst game ever

Result 2
Review Type: Negative review
Review Topic: ComputerGame
Score: 1.0
Review Text: Worst game ever

Result 3
Review Type: Negative review
Review Topic: ComputerGame
Score: 1.0
Review Text: Worst game ever

Result 4
Review Type: Negative review
Review Topic: ComputerGame
Score: 1.0
Review Text: Worst game ever

Result 5
Review Type: Negative review
Review Topic: ComputerGame
Score: 1.0
Review Text: Worst game ever


## STEP 4 — CONVERSATIONAL ORCHESTRATION

In [ ]:
%pip install autogen

In [ ]:
%pip install autogen-agentchat autogen-ext sentence-transformers chromadb

In [50]:
import os
import asyncio
from dotenv import load_dotenv
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

# Load environment variables
load_dotenv()
groq_api_key = os.getenv('GROQCLOUD_API_KEY')

if not groq_api_key:
    raise ValueError("GROQCLOUD_API_KEY not found in .env file")

In [61]:
groq_model_info = {
    "vision": False,
    "function_calling": True,
    "json_output": True,
    "family": "unknown",
    "structured_output": False
}

model_client = OpenAIChatCompletionClient(
    model="llama-3.3-70b-versatile",
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1",
    model_info=groq_model_info
)

# Utility / lightweight model
utility_model_client = OpenAIChatCompletionClient(
    model="llama-3.1-8b-instant",
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1",
    model_info=groq_model_info
)

# Reasoning / decision model
reasoning_model_client = OpenAIChatCompletionClient(
    model="llama-3.3-70b-versatile",
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1",
    model_info=groq_model_info,
)

In [62]:
def retrieve_reviews(query: str, top_k: int = 8) -> str:
    """Retrieve relevant reviews with deduplication"""
    results = negative_vectorstore.similarity_search(query, k=top_k * 2)

    seen = set()
    unique_reviews = []

    for doc in results:
        text = doc.page_content.strip()
        if text not in seen:
            seen.add(text)
            unique_reviews.append(text)
        if len(unique_reviews) >= top_k:
            break

    formatted = "\n\n---\n\n".join([
        f"Review {i+1}:\n{text}"
        for i, text in enumerate(unique_reviews)
    ])

    return (
        f"Found {len(unique_reviews)} unique complaint reviews "
        f"(deduplicated from {len(results)} retrieved):\n\n{formatted}"
    )


In [69]:
## STEP 5: Define Agents

retriever_agent = AssistantAgent(
    name="Retriever",
    model_client=utility_model_client,
    system_message="""You are a review retrieval specialist.
    Use the retrieve_reviews tool to find relevant customer reviews.
    Present the reviews clearly without analyzing them.""",
    tools=[retrieve_reviews],
)

analyzer_agent = AssistantAgent(
    name="Analyzer",
    model_client=reasoning_model_client,
    system_message="""You are a sentiment analysis expert.
    Analyze the reviews provided by the Retriever.
    Identify:
    - Overall sentiment (positive/negative/neutral)
    - Key themes and patterns
    - Specific pain points
    - Severity of issues""",
)

categorizer_agent = AssistantAgent(
    name="Categorizer",
    model_client=utility_model_client,
    system_message="""You categorize customer complaints into themes:
    - Performance Issues (lag, crashes, freezes)
    - Monetization (ads, pay-to-win, pricing)
    - Technical Problems (bugs, battery drain)
    - UI/UX Problems (confusing interface, navigation)
    - Customer Service
    
    Provide counts and priority levels for each category.""",
)

summarizer_agent = AssistantAgent(
    name="Summarizer",
    model_client=utility_model_client,
    system_message="""You create executive summaries with:
    - Top 3 critical issues
    - Overall sentiment breakdown
    - Actionable recommendations
    - Priority rankings""",
)

decision_agent = AssistantAgent(
    name="DecisionMaker",
    model_client=reasoning_model_client,
    system_message="""
        You are a Lead Business Analyst.

        Your task has TWO phases:

        PHASE 1 — THINKING (PRIVATE)
        - Carefully read all information from previous agents
        - List key evidence points
        - Check if evidence is sufficient
        - Identify uncertainties or gaps
        - Decide conservatively if evidence is weak

        PHASE 2 — FINAL OUTPUT (PUBLIC)
        - Output STRICT JSON ONLY
        - Do NOT include explanations outside JSON
        - Do NOT mention other agents
        - Base everything ONLY on retrieved reviews
        - If evidence is limited, lower confidence_level

        IMPORTANT FORMAT RULES:
        - First output a <THINKING>...</THINKING> block
        - Then output <FINAL_JSON>...</FINAL_JSON>
        - NOTHING after FINAL_JSON

        Required JSON schema:

        {
        "topic": "<review topic>",
        "analysis_scope": "<e.g. Negative reviews only>",
        "evidence_count": <number>,
        "confidence_level": "<High | Medium | Low>",
        "primary_issue": "<main problem identified>",
        "evidence": [
            "<short quote or paraphrase from review>"
        ],
        "root_causes": [
            "<cause 1>",
            "<cause 2>"
        ],
        "business_impact": [
            "<impact 1>",
            "<impact 2>"
        ],
        "recommended_actions": [
            "<action 1>",
            "<action 2>"
        ]
        }
        """,
)

critic_agent = AssistantAgent(
    name="Critic",
    model_client=reasoning_model_client,
    system_message="""
        You are a strict validation agent.

        Your ONLY task is to validate the DecisionMaker's FINAL_JSON.

        Rules:
        - NEVER output JSON.
        - NEVER repeat the DecisionMaker output.
        - NEVER add new facts.

        Validation checklist:
        1. evidence_count must equal the number of items in "evidence".
        2. Confidence calibration:
        - evidence_count = 1 → confidence must be Low
        - evidence_count ≤ 2 → confidence cannot be High
        - evidence_count ≥ 10 → confidence should NOT be Low
        3. Evidence statements must be supported by retrieved reviews.
        4. No new facts allowed.

        Output format (STRICT, ONE OF TWO):

        If valid:
        APPROVED

        If invalid:
        ERROR:
        - <one-line explanation>
        - <what must be changed>

        Do not output anything else.
        """
)



team = RoundRobinGroupChat(
    participants=[
        retriever_agent,
        analyzer_agent,
        categorizer_agent,
        summarizer_agent,
        decision_agent,
        critic_agent,
    ],
    termination_condition=MaxMessageTermination(max_messages=6),
)

In [74]:
async def run_analysis(query: str):
    print(f"\n{'='*80}")
    print("QUERY:")
    print(f"{'='*80}\n")

    # First run
    result = await team.run(task=query)

    decision_json = None
    critic_verdict = None

    for msg in result.messages:
        if msg.source == "DecisionMaker":
            decision_json = msg.content
        elif msg.source == "Critic":
            critic_verdict = msg.content

    # If approved, finish
    if critic_verdict and critic_verdict.strip() == "APPROVED":
        print(f"\n{'='*80}")
        print("FINAL DECISION (APPROVED)")
        print(f"{'='*80}\n")
        print(decision_json)
        return result

    # If error, allow ONE correction
    if critic_verdict and critic_verdict.startswith("ERROR"):
        print(f"\n{'='*80}")
        print("CRITIC FOUND ISSUES — AUTO-CORRECTING ONCE")
        print(f"{'='*80}\n")
        print(critic_verdict)

        correction_prompt = f"""
The Critic has found issues in your previous decision:

{critic_verdict}

Please FIX the issues and output a corrected FINAL_JSON.
Follow the same schema.
Do not add new evidence.
"""

        # Second run: ONLY DecisionMaker + Critic
        correction_team = RoundRobinGroupChat(
            participants=[decision_agent, critic_agent],
            termination_condition=MaxMessageTermination(max_messages=2),
        )

        corrected_result = await correction_team.run(task=correction_prompt)

        corrected_decision = None
        corrected_verdict = None

        for msg in corrected_result.messages:
            if msg.source == "DecisionMaker":
                corrected_decision = msg.content
            elif msg.source == "Critic":
                corrected_verdict = msg.content

        print(f"\n{'='*80}")
        print("FINAL DECISION (AFTER CORRECTION)")
        print(f"{'='*80}\n")
        print(corrected_decision)

        print(f"\n{'='*80}")
        print("FINAL CRITIC VERDICT")
        print(f"{'='*80}\n")
        print(corrected_verdict)

        return corrected_result


In [75]:
queries ="""
        Retrieve relevant customer reviews about Computer game complaints.
        Analyze them to identify recurring issues.
        Categorize the issues.
        Then produce ONE final business summary with recommendations.
        """

# Run first query
result = await run_analysis(queries)


QUERY:


FINAL DECISION (APPROVED)

<THINKING> 
The customer reviews about the computer game are overwhelmingly negative, with concerns about game quality, pricing, and mobile data usage. 
The analysis highlights specific pain points, including poor game quality, expensive pricing, and high mobile data usage. 
The evidence is sufficient to support the conclusion that the game requires significant improvements to address customer concerns. 
The primary issues identified are poor game quality, expensive pricing, and high mobile data usage. 
The root causes of these issues are likely related to game development, pricing strategy, and data optimization. 
The business impact of these issues is significant, with potential consequences including negative word-of-mouth, decreased customer satisfaction, and reduced sales. 
The recommended actions are to reinvest in game development, optimize mobile data usage, and price competitively. 
</THINKING> 

<FINAL_JSON>
{
  "topic": "Computer Game Com